<a href="https://colab.research.google.com/github/fastinom/fassy/blob/main/wordprediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Fastino Mateteva
R205684K**

In [ ]:

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import pandas as pd
import os 

In [ ]:
 from google.colab import files
uploaded= files.upload()

Load and Pre-process the data

In [ ]:
file = open("txt.txt", "r", encoding = "utf8")
#store file in list
lines =[]
for i in file:
  lines.append(i)

#Convert list to string 
data = ""
for i in lines:
  data = ' '.join(lines)
#replace unneccessary stuff with space
data = data.replace('\n','').replace('\ufeff','').replace('"','').replace('"','') #new line, carriage return, unicode character

#rmove unnecessary spaces 
data = data.split()
data =' '.join(data)
data[:500]

In [ ]:
len(data)

Apply Tokenization

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

#saving the tokenizer for predict function
pickle.dump(tokenizer,open('token.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data]) [0]
sequence_data[:15]

In [ ]:
len(sequence_data)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

In [ ]:
sequences = []

for i in range(5, len(sequence_data)):
  words = sequence_data[i-5:i+1]
  sequences.append(words)

print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences [:10]

In [ ]:
x = []
y = []
for i in sequences:
   x.append(i[0:5])
   y.append(i[5])
x = np.array(x)
y = np.array(y)

In [ ]:
print("Data: ", x[:10])
print("Response: ", y[:10])

In [ ]:

y = to_categorical(y, num_classes=vocab_size)
y[:5]

CREATING THE MODEL

In [ ]:
from keras.models import Sequential
model = Sequential ()
model.add(Embedding(vocab_size, 10, input_length=5))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

In [ ]:
model.summary()

In [ ]:
from tensorflow import keras 
from keras.utils.vis_utils import plot_model

keras.utils.plot_model(model, to_file='plot.png', show_layer_names=True)

**Building a model **

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard

checkpoint = ModelCheckpoint("nextword.h5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto')

reduce = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001, verbose = 1)

logdir='logsnextword1'
tensorboard_Visualization = TensorBoard(log_dir=logdir)

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.002), metrics=['accuracy'])

In [ ]:
history=model.fit(x, y, epochs=75, batch_size=64, callbacks=[checkpoint, reduce, tensorboard_Visualization])

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.show()

In [ ]:

from tensorflow.keras.models import load_model
import numpy as np
import pickle

# Load the model and tokenizer
#model = load_model('next_words.h5')
#tokenizer = pickle.load(open('token.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):

  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predicted_word = ""
  
  for key, value in tokenizer.word_index.items():
      if value == preds:
          predicted_word = key
          break
  
  print(predicted_word)
  return predicted_word

In [ ]:
while(True):
  text = input("Enter your line: ")
  
  if text == "0":
      print("Execution completed.....")
      break
  
  else:
      try:
          text = text.split(" ")
          text = text[-5:]
          print(text)
        
          Predict_Next_Words(model, tokenizer, text)
          
      except Exception as e:
        print("Error occurred: ",e)
        continue